In [18]:
import weaviate
import os
client = weaviate.Client(
    "http://localhost:8082",
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")
    }
)
client.schema.delete_all()
client.schema.get()

{'classes': []}

In [19]:
import weaviate.weaviate_classes as wvc

cat_collection_config = wvc.CollectionConfig(
    name="JeopardyCategory",
    description="A Jeopardy! category",
    properties=[
        wvc.Property(
            name="title",
            dataType=wvc.DataType.TEXT,
            tokenization=wvc.Tokenization.FIELD
        ),
    ],
    vectorizer=wvc.Vectorizer.TEXT2VEC_OPENAI,

    # QUESTION FOR DIRK - are invertedIndexConfig and vectorIndexConfig availble yet?
)

categories = client.collection.create(cat_collection_config)

In [20]:
cat_uuid = categories.data.insert(
    {"title": "Famous people"}
)
print(cat_uuid)

70ce1fe3-7eba-44e1-b22a-6b4b6da8f93d


In [21]:
categories.query.get_flat(limit=2)

AttributeError: 'NoneType' object has no attribute 'Search'

## Hypothetical Filtering Syntax

```python
from weaviate.weaviate_classes import FilterOperator

articles.query.get(
    limit=2,
    filter=Filter(
        path=["page"],
        operator=FilterOperator.GREATERTHAN,  # Enum
        value=10,
    )
)
```

Nested Filter

```python
from weaviate.weaviate_classes import FilterOperator

articles.query.get(
    limit=2,
    filter=Filter(
        operator=FilterOperator.AND,  # Enum
        operands=[
            Filter(
                path=["pages"],
                operator=FilterOperator.GREATERTHAN,  # Enum
                value=10,
            ),
            Filter(
                path=["title"],
                operator=FilterOperator.LIKE,  # Enum
                value="*football*",
            ),
        ]
    )
)
```


Nested Filter

```python
from weaviate.weaviate_classes import FilterOperator

articles.query.get(
    limit=2,
    filter=Filter(
        # path=["isAuthor", "Author", "name"],  # Option 1
        # path=ReferenceProperty(name="isAuthor", ref_prop="name"),
        ReferenceProperty(name="isAuthor", ref_collection="Author", ref_prop="name"),
        operator=FilterOperator.GREATERTHAN,  # Enum
        value=10,
    )
)
```

GeoCoordinates

```python
from weaviate.weaviate_classes import FilterOperator, valueGeoRange

publications.query.get(
    limit=2,
    filter=Filter(
        path=["headquartersGeoLocation"],
        operator=FilterOperator.WithinGeoRange,  # Enum
        value=valueGeoRange(
            latitude=51.51,
            longitude=-0.09,
            distance_max=2000,
        ),
    ),
    return_properties=[
        "name",
        "headquartersGeoLocation {lat} {long}"  # this is nasty atm
    ]
)
```


## Hypothetical Sorting Syntax

```python
import weaviate.weaviate_classes as wvc

articles.query.get(
    limit=2,
).sort_results(
    by=["title"],
    order=wvc.SortOrder.ASCENDING
)
```

```python
import weaviate.weaviate_classes as wvc

articles.query.get(
    limit=2,
).sort_results(
    by=ReferenceProperty(name="isAuthor", ref_prop="name"),
    order=wvc.SortOrder.ASCENDING
)
```
